<a href="https://colab.research.google.com/github/gvogiatzis/CS4740/blob/main/CS4740_Lab_Week_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS4740 Labs
##Week 4 - Convolutional Neural Networks

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

train_dataset = torchvision.datasets.CIFAR10('.', train=True, download=True,
                             transform=torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.CIFAR10('.', train=False, download=True,
                             transform=torchvision.transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 64, 5)
        self.conv3 = nn.Conv2d(64, 256, 5)
        self.conv4 = nn.Conv2d(256, 128, 1)
        self.conv5 = nn.Conv2d(128, 10, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.conv5(x)
        return x.view(-1,10)

convnet = ConvNet()
convnet = convnet.cuda()

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1000, shuffle=True)

In [ ]:
x,t = next(iter(train_loader))
x,t=x.cuda(),t.cuda()
convnet(x).shape

torch.Size([64, 10])

In [ ]:
def train(net,dataloader, optimizer,loss_fun):
  net.train()
  for x, t in dataloader:
    x = x.cuda()
    t = t.cuda()
    optimizer.zero_grad()
    L = loss_fun(net(x), t)
    L.backward()
    optimizer.step()

def test(net,dataloader,loss_fun):
  net.eval()
  total_L = 0
  correct = 0
  with torch.no_grad():
    for x, t in dataloader:
      x = x.cuda()
      t = t.cuda()
      out = net(x)
      total_L += loss_fun(out, t)
      _,pred = out.max(dim=1)     # this counts how many we got right
      correct += (pred==t).sum() 
  total_L /= len(dataloader.dataset)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    total_L, correct, len(dataloader.dataset),
    100. * correct / len(dataloader.dataset)))

In [ ]:
optim_SGD = optim.SGD(convnet.parameters(), lr=0.01, momentum=0.5)

for e in range(100):
  print(f"Epoch: {e+1}/10. Training ...")
  train(convnet, train_loader, optim_SGD, nn.CrossEntropyLoss())
  print("Testing ...")
  test(convnet, test_loader,  nn.CrossEntropyLoss(reduction='sum'))  

Epoch: 1/10. Training ...
Testing ...

Test set: Avg. loss: 2.2167, Accuracy: 1771/10000 (18%)

Epoch: 2/10. Training ...
Testing ...

Test set: Avg. loss: 1.9338, Accuracy: 2952/10000 (30%)

Epoch: 3/10. Training ...
Testing ...

Test set: Avg. loss: 1.6991, Accuracy: 3991/10000 (40%)

Epoch: 4/10. Training ...
Testing ...

Test set: Avg. loss: 1.4462, Accuracy: 4780/10000 (48%)

Epoch: 5/10. Training ...
Testing ...

Test set: Avg. loss: 1.4992, Accuracy: 4779/10000 (48%)

Epoch: 6/10. Training ...
Testing ...

Test set: Avg. loss: 1.4133, Accuracy: 4910/10000 (49%)

Epoch: 7/10. Training ...
Testing ...

Test set: Avg. loss: 1.2807, Accuracy: 5479/10000 (55%)

Epoch: 8/10. Training ...
Testing ...

Test set: Avg. loss: 1.2378, Accuracy: 5589/10000 (56%)

Epoch: 9/10. Training ...
Testing ...

Test set: Avg. loss: 1.1337, Accuracy: 5984/10000 (60%)

Epoch: 10/10. Training ...
Testing ...

Test set: Avg. loss: 1.0972, Accuracy: 6088/10000 (61%)

Epoch: 11/10. Training ...
Testing ...
